# Week 7 - Class 7

This notebook have the most important exercises for Chapter 7.
The lab 7 from https://github.com/fchollet/deep-learning-with-python-notebooks is a guide for this notebook.

It will be covered:
- 3 ways to build keras models: sequential, funtional api, Multi-input, multi-output models
- Using middle layers
- Writting your own metrics
- Callbacks
- TensorBoard


In [ ]:
#perform all your imports in the beginning 
from tensorflow import keras
from tensorflow.keras import layers

# Sequential 
The weights are defined when the input is defined

In [ ]:
# the sequential class
model = keras.Sequential([
    layers.Dense(64, activation="relu"),
    layers.Dense(10, activation="softmax")
])

In [ ]:
# this will give an error because the NN is not yet initialized 
model.weights

In [ ]:
# this can be increamental, adding layer by layer
model = keras.Sequential()
model.add(layers.Dense(64, activation="relu"))
model.add(layers.Dense(10, activation="softmax"))

In [ ]:
# the build method calls and initializes the weights of the layers (randomly)
model.build(input_shape=(None, 3))
model.weights

input_shape(728,) is the same as batch_input=(batch_size,728) or input_shape=(None, 728)

1.1 Call the summary method to see the layers

In [ ]:
<your-answer>

In [ ]:
# Another way to initialize the weights without the build method, instead you spoecify the input
model2 = keras.Sequential()
model2.add(keras.Input(shape=(3,)))
model2.add(layers.Dense(64, activation="relu"))
model2.add(layers.Dense(10, activation="softmax"))

1.2 Check the weights values of model2

In [ ]:
<your-answer>

# Funtional 

In [ ]:
inputs = keras.Input(shape=(3,), name="my_input")
features = layers.Dense(64, activation="relu")(inputs)
outputs = layers.Dense(10, activation="softmax")(features)
modelF = keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
modelF.summary()

see the model

In [ ]:
keras.utils.plot_model(modelF, "ticket_classifier.png")

In [ ]:
keras.utils.plot_model(modelF, "ticket_classifier_with_shape_info.png", show_shapes=True)

Let's draw in the class this model as a NN in the graphical perspective 

# Multi 

Let’s say you’re building a system to rank customer support tickets by priority and route them to the appropriate department. Your model has three inputs:
- The title of the ticket (text input)
- The text body of the ticket (text input)
- Any tags added by the user (categorical input, assumed here to be one-hot encoded)

We can encode the text inputs as arrays of ones and zeros of size vocabulary_size (see chapter 11 for detailed information about text encoding techniques). Your model also has two outputs:
- The priority score of the ticket, a scalar between 0 and 1 (sigmoid output)
- The department that should handle the ticket (a softmax over the set of departments)

Create a new model by reusing intermediate layer outputs

1.3 

The total number of diferent letters in all text features is 1000, the possible categorical input has 100 classes and there are 4 departments in this customer support ticket system.

What are the following values of these variables:

In [ ]:
import numpy as np

vocabulary_size = <your-answer>
num_tags = <your-answer>
num_departments = <your-answer>

In [ ]:
# defining the multi NN with the functional pattern in keras

# inputs
title = keras.Input(shape=(None,vocabulary_size), name="title")
text_body = keras.Input(shape=(None,vocabulary_size), name="text_body")
tags = keras.Input(shape=(None,num_tags), name="tags")
# hidden layers
features = layers.Concatenate()([title, text_body, tags])
features = layers.Dense(64, activation="relu")(features)
# output
priority = layers.Dense(1, activation="sigmoid", name="priority")(features)
department = layers.Dense(
    num_departments, activation="softmax", name="department")(features)
# model
model = keras.Model(inputs=[title, text_body, tags], outputs=[priority, department])

In [ ]:
# create random data

num_samples = 128

title_data = np.random.randint(0, 2, size=(num_samples, vocabulary_size))
text_body_data = np.random.randint(0, 2, size=(num_samples, vocabulary_size))
tags_data = np.random.randint(0, 2, size=(num_samples, num_tags))

priority_data = np.random.random(size=(num_samples, 1))
department_data = np.random.randint(0, 2, size=(num_samples, num_departments))

In [ ]:
#investigate the input data

1.4 

Choose the right metric for the department label

In [ ]:
model.compile(optimizer="rmsprop",
              loss=["mean_squared_error", "categorical_crossentropy"],
              metrics=[["mean_absolute_error"], [<your-answer>]])
model.fit([title_data, text_body_data, tags_data],
          [priority_data, department_data],
          epochs=1)
model.evaluate([title_data, text_body_data, tags_data],
               [priority_data, department_data])
priority_preds, department_preds = model.predict([title_data, text_body_data, tags_data])

In [ ]:
keras.utils.plot_model(model, "ticket_classifier.png")

The “None” in the tensor shapes represents the batch size: this model allows batches of any size.

In [ ]:
keras.utils.plot_model(model, "ticket_classifier_with_shape_info.png", show_shapes=True)

In [ ]:
model.layers

1.5

See the input and output of the 4th layer

In [ ]:
<your-answer>

# Using middle layers

In [ ]:
#using previous layers
features = model.layers[4].output

#adding another output
difficulty = layers.Dense(3, activation="softmax", name="difficulty")(features)

new_model = keras.Model(
    inputs=[title, text_body, tags],
    outputs=[priority, department, difficulty])

In [ ]:
keras.utils.plot_model(new_model, "updated_ticket_classifier.png", show_shapes=True)

# Writing own metrics

The standard workflow: compile(), fit(), evaluate(), predict()

In [ ]:
from tensorflow.keras.datasets import mnist

def get_mnist_model():
    inputs = keras.Input(shape=(28 * 28,))
    features = layers.Dense(512, activation="relu")(inputs)
    features = layers.Dropout(0.5)(features)
    outputs = layers.Dense(10, activation="softmax")(features)
    model = keras.Model(inputs, outputs)
    return model

(images, labels), (test_images, test_labels) = mnist.load_data()
images = images.reshape((60000, 28 * 28)).astype("float32") / 255
test_images = test_images.reshape((10000, 28 * 28)).astype("float32") / 255
train_images, val_images = images[10000:], images[:10000]
train_labels, val_labels = labels[10000:], labels[:10000]

model = get_mnist_model()
model.compile(optimizer="rmsprop",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])
model.fit(train_images, train_labels,
          epochs=3,
          validation_data=(val_images, val_labels))
test_metrics = model.evaluate(test_images, test_labels)
predictions = model.predict(test_images)

In [ ]:
import tensorflow as tf

class RootMeanSquaredError(keras.metrics.Metric):

    def __init__(self, name="rmse", **kwargs):
        super().__init__(name=name, **kwargs)
        self.mse_sum = self.add_weight(name="mse_sum", initializer="zeros")
        self.total_samples = self.add_weight(
            name="total_samples", initializer="zeros", dtype="int32")

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.one_hot(y_true, depth=tf.shape(y_pred)[1])
        mse = tf.reduce_sum(tf.square(y_true - y_pred))
        self.mse_sum.assign_add(mse)
        num_samples = tf.shape(y_pred)[0]
        self.total_samples.assign_add(num_samples)

    def result(self):
        return tf.sqrt(self.mse_sum / tf.cast(self.total_samples, tf.float32))

    def reset_state(self):
        self.mse_sum.assign(0.)
        self.total_samples.assign(0)

In [ ]:
model = get_mnist_model()
model.compile(optimizer="rmsprop",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy", RootMeanSquaredError()])
model.fit(train_images, train_labels,
          epochs=3,
          validation_data=(val_images, val_labels))
test_metrics = model.evaluate(test_images, test_labels)

# Callbacks

In [ ]:
callbacks_list = [
    keras.callbacks.EarlyStopping(
        monitor="val_accuracy",
        patience=2,
    ),
    keras.callbacks.ModelCheckpoint(
        filepath="checkpoint_path.keras",
        monitor="val_loss",
        save_best_only=True,
    )
]
model = get_mnist_model()
model.compile(optimizer="rmsprop",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])
model.fit(train_images, train_labels,
          epochs=10,
          callbacks=callbacks_list,
          validation_data=(val_images, val_labels))

In [ ]:
model = keras.models.load_model("checkpoint_path.keras")

custom callbacks

In [ ]:
from matplotlib import pyplot as plt

class LossHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs):
        self.per_batch_losses = []

    def on_batch_end(self, batch, logs):
        self.per_batch_losses.append(logs.get("loss"))

    def on_epoch_end(self, epoch, logs):
        plt.clf()
        plt.plot(range(len(self.per_batch_losses)), self.per_batch_losses,
                 label="Training loss for each batch")
        plt.xlabel(f"Batch (epoch {epoch})")
        plt.ylabel("Loss")
        plt.legend()
        plt.savefig(f"plot_at_epoch_{epoch}")
        self.per_batch_losses = []

In [ ]:
model = get_mnist_model()
model.compile(optimizer="rmsprop",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])
model.fit(train_images, train_labels,
          epochs=10,
          callbacks=[LossHistory()],
          validation_data=(val_images, val_labels))

# TensorBoard

In [ ]:
model = get_mnist_model()
model.compile(optimizer="rmsprop",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

tensorboard = keras.callbacks.TensorBoard(
    log_dir="/full_path_to_your_log_dir",
)
model.fit(train_images, train_labels,
          epochs=10,
          validation_data=(val_images, val_labels),
          callbacks=[tensorboard])

In [ ]:
%load_ext tensorboard
%tensorboard --logdir /full_path_to_your_log_dir